In [3]:
import numpy as np
from scipy.stats import norm

#BS Price
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma **2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price

In [5]:
#Fake Options Portfolio Data
portfolio = [
    {'position': 10, 'type': 'call', 'strike': 400, 'T': 0.5},  # Long 10 calls
    {'position': -5, 'type': 'put',  'strike': 350, 'T': 0.5}   # Short 5 puts
]

In [14]:
#Monte Carlo Params [********CONFIGURABLE********]
S0 = 380
r = 0.01
sigma = 0.25
simulations = 100000
confidence = 0.95

#Perform MC Sims using GBM

np.random.seed(42)
ST = S0 * np.exp((r - 0.5 * sigma**2) * 0.5 + sigma * np.sqrt(0.5) * np.random.randn(simulations))

In [9]:
initial_value = 0
for opt in portfolio:
    price = black_scholes_price(S0, opt['strike'], opt['T'], r, sigma, opt['type'])
    initial_value += opt['position'] * price

#Simulated portfolio value at expiry
simulated_values = np.zeros(simulations)

for opt in portfolio:
    if opt['type'] == 'call':
        payoff = np.maximum(ST - opt['strike'], 0)
    else:
        payoff = np.maximum(opt['strike'] - ST, 0)
        
    simulated_values += opt['position'] * payoff

PnL = simulated_values - initial_value

In [10]:
PnL

array([ -24.24308652, -129.15199883,   86.78684786, ...,  -87.26988744,
       -129.15199883, -129.15199883], shape=(100000,))

In [11]:
sum(PnL)

np.float64(129769.94553449216)

In [12]:
#sort from worst to best
sorted_losses = -np.sort(-PnL)

VaR_idx = int((1 - confidence) * simulations)
VaR = -sorted_losses[VaR_idx]

CVaR = -np.mean(sorted_losses[:VaR_idx])

In [13]:
print(f"Portfolio VaR ({int(confidence*100)}%): ${VaR:,.2f}")
print(f"Portfolio CVaR ({int(confidence*100)}%): ${CVaR:,.2f}")

Portfolio VaR (95%): $-904.52
Portfolio CVaR (95%): $-1,297.35
